# LFP processing of Indefinite Streaming

In [2]:
# Importing Python and external packages


import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

import scipy
from scipy import stats
from scipy import signal
from scipy.signal import spectrogram, butter, filtfilt, freqz
from scipy.signal.windows import hann

import seaborn as sns
import pingouin as pg
from itertools import combinations
#from statannotations.Annotator import Annotator

import openpyxl
from openpyxl import Workbook, load_workbook
# import xlrd
import pickle
import json
import csv
import fooof
from fooof.plts.spectra import plot_spectrum

#mne
import mne
from mne.time_frequency import tfr_morlet 

# xdf
import pyxdf
import pyxdftools
from pyxdftools import XdfData

import datetime

/var/folders/8d/3m_ljz890_j62q7hb7mpzdc40000gn/T/ipykernel_52934/2413707706.py:32: DeprecationWarning: 
The `fooof` package is being deprecated and replaced by the `specparam` (spectral parameterization) package.
This version of `fooof` (1.1) is fully functional, but will not be further updated.
New projects are recommended to update to using `specparam` (see Changelog for details).
  import fooof


In [25]:
%matplotlib qt


In [42]:
jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Patterned_stimulation_project', 'patterned_DBS')
sys.path.append(project_path)

os.chdir(project_path)


# data processing
import src.patterned_DBS.data_processing.indefinite_streaming as indef_streaming

# utility functions
import src.patterned_DBS.utils.io as io
import src.patterned_DBS.utils.find_folders as find_folders
import src.patterned_DBS.utils.lfp_preprocessing as lfp_preprocessing
import src.patterned_DBS.utils.xdf_structure as xdf_structure

importlib.reload(indef_streaming)
importlib.reload(io)
importlib.reload(find_folders)
importlib.reload(lfp_preprocessing)
importlib.reload(xdf_structure)

<module 'src.patterned_DBS.utils.xdf_structure' from '/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/xdf_structure.py'>

### 1. Load indefinite streaming (StimOFF), structure the data

In [14]:
indef_str_data = indef_streaming.structure_indef_str(
    sub="084",
    medication="on",
    stimulation="burst",
    run="1",
)

Creating RawArray with float64 data, n_channels=12, n_times=49125
    Range : 0 ... 49124 =      0.000 ...   196.496 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess th

In [15]:
indef_str_data["data_info"]

,sub,medication,stimulation,run,fs,n_channels,n_samples_original,duration_original,n_samples_kept,duration_kept
0,084,on,burst,1,250.0,12,49125,196.5,45000,180.0


In [16]:
indef_str_data["raw_data"]

,LFP_Stn_L_03,LFP_Stn_L_13,LFP_Stn_L_02,LFP_Stn_R_03,LFP_Stn_R_13,LFP_Stn_R_02,LFP_Stn_L_01,LFP_Stn_L_12,LFP_Stn_L_23,LFP_Stn_R_01,LFP_Stn_R_12,LFP_Stn_R_23
0,-11.965908,-24.712635,-14.761681,-11.854076,-13.755202,-1.440993,-12.746727,-27.508408,2.795773,-1.901126,-3.342119,-10.413083
1,-9.281965,-16.175543,-17.221960,-16.662806,-19.234917,-3.436214,-6.893578,-24.115538,7.939995,-2.572111,-6.008325,-13.226592
2,-5.815207,-13.142629,-17.669285,-17.781115,-25.050125,-3.103677,-7.327421,-24.996706,11.854077,-7.269010,-10.372687,-14.677438
3,-0.223662,-7.526121,-13.531540,-20.576887,-30.306177,-5.763972,-7.302459,-20.833999,13.307878,-9.729290,-15.493262,-14.812915
4,-1.677464,-4.268546,-10.959430,-21.471535,-26.727589,-4.322979,-2.591082,-13.550512,9.281966,-5.256054,-9.579033,-17.148556
...,...,...,...,...,...,...,...,...,...,...,...,...
44995,17.781115,5.616508,18.340270,3.354927,11.742246,-15.850924,-12.164607,6.175663,-0.559155,8.387319,-7.463605,19.205851
44996,18.563930,18.983797,11.854076,10.959430,19.123087,-11.971327,0.419867,12.273943,6.709854,8.163657,-3.807670,22.930757
44997,19.346748,24.600304,11.183091,16.662806,24.490970,-8.867649,5.253556,16.436647,8.163657,7.828164,-1.039485,25.530455
44998,18.228440,26.172926,8.163656,21.359705,31.200825,-4.212133,7.944486,16.108142,10.064784,9.841120,5.628987,25.571838


### 2. Only pick the channels around the stimulation contacts (02 or 13)

In [17]:
picked_channels = indef_streaming.pick_channel(
    sub="084",
    medication="on",    
    stimulation="burst",
    run="1",
)

Excel file loaded:  metadata_084.xlsx 
loaded from:  /Users/jenniferbehnke/OneDrive - Charité - Universitätsmedizin Berlin/Burst_DBS_project - AG Bewegungsstörungen/data/sub-084
Creating RawArray with float64 data, n_channels=12, n_times=49125
    Range : 0 ... 49124 =      0.000 ...   196.496 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess th

In [18]:
picked_channels["picked_channels"]

,Left_02,Right_13
0,-14.761681,-13.755202
1,-17.221960,-19.234917
2,-17.669285,-25.050125
3,-13.531540,-30.306177
4,-10.959430,-26.727589
...,...,...
44995,18.340270,11.742246
44996,11.854076,19.123087
44997,11.183091,24.490970
44998,8.163656,31.200825


In [19]:
picked_channels["data_info"]

,sub,medication,stimulation,run,fs,n_channels,n_samples_original,duration_original,n_samples_kept,duration_kept
0,084,on,burst,1,250.0,12,49125,196.5,45000,180.0


### 3. Plot raw time series, check for artifacts

In [43]:
plot_raw_time_series = indef_streaming.plot_raw_time_series(
    sub="084",
    medication="on",    
    stimulation="burst",
    run="1",
    channels="all"
)

Creating RawArray with float64 data, n_channels=12, n_times=49125
    Range : 0 ... 49124 =      0.000 ...   196.496 secs
Ready.


/Users/jenniferbehnke/anaconda3/envs/burst_dbs/lib/python3.12/site-packages/pymatreader/utils.py:291: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(file_path, info={}, data_name="data")
/Users/jenniferbehnke/code/Patterned_stimulation_project/patterned_DBS/src/patterned_DBS/utils/io.py:212: RuntimeWarning: Cannot guess th